In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir ./drive/MyDrive/vtb24_hktn

mkdir: cannot create directory ‘./drive/MyDrive/vtb24_hktn’: File exists


In [ ]:
!ls ./drive/MyDrive/vtb24_hktn

atms.csv  atms.gsheet  offices.csv  offices.gsheet


In [38]:
import numpy as np
import pandas as pd
import datetime
from pandas.core.arrays import boolean
from tqdm.auto import tqdm
import random

In [6]:
offices_excel = pd.read_csv('./drive/MyDrive/vtb24_hktn/offices.csv')

In [7]:
df_loading = pd.DataFrame(columns=['dateTime', 'salePointName', 'status', 'clients', 'employers'], index=['dateTime'])
df_offices = pd.DataFrame(columns=['salePointName', 'status', 'employers', 'morning_k', 'daily_peak_k', 'dinner_k', 'evening_peak_k', 'evining_k'])

In [8]:
for office in offices_excel['sale_point_name']:
  # print(office)
  df_offices = pd.concat([pd.DataFrame([[office, True, random.randint(5,25),
    random.randint(-15, -5), random.randint(5, 15), random.randint(-20, -10),
    random.randint(10, 20), random.randint(-25, -5)]], columns=df_offices.columns), df_offices], ignore_index=True)
df_offices

<ipython-input-8-93941cfdb499>:3: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_offices = pd.concat([pd.DataFrame([[office, True, random.randint(5,25),


,salePointName,status,employers,morning_k,daily_peak_k,dinner_k,evening_peak_k,evining_k
0,ДО «ЗИО Подольск» Филиала № 7701 Банка ВТБ (ПАО),True,8,-11,7,-12,20,-11
1,ДО «Бронницкий» Филиала № 7701 Банка ВТБ (ПАО),True,20,-9,5,-17,20,-12
2,ДО «Московский» в г. Московском Филиала № 7701...,True,20,-15,15,-13,15,-12
3,ДО «Центральный» в г. Подольске Филиала № 7701...,True,13,-8,7,-15,19,-15
4,ДО «Подольский» Филиала № 7701 Банка ВТБ (ПАО),True,8,-7,8,-20,19,-5
...,...,...,...,...,...,...,...,...
273,ДО «Пушкинский» Филиала № 7701 Банка ВТБ (ПАО),True,13,-6,8,-16,20,-15
274,ДО «Черноголовский» Филиала № 7701 Банка ВТБ (...,True,14,-12,7,-17,16,-13
275,ДО «Лобня» Филиала № 7701 Банка ВТБ (ПАО),True,21,-14,13,-15,11,-14
276,ДО «На Баранова» Филиала № 7701 Банка ВТБ (ПАО),True,18,-6,14,-20,13,-25


In [9]:
df_offices.drop_duplicates(subset=['salePointName'], inplace=True)

In [10]:
def one_row(dateTime: datetime, salePointName: str, status: boolean, clients: int, employers: int):
  global df_loading
  df_loading = pd.concat([pd.DataFrame([[dateTime, salePointName, status, clients, employers]], columns=df_loading.columns), df_loading], ignore_index=True)
  return

In [39]:
date_time_i = datetime.datetime(2022, 1, 3, 8, 00, 00)
date_time_end = datetime.datetime(2022, 1, 18, 0, 00, 00)
df_loading = pd.DataFrame(columns=['dateTime', 'salePointName', 'status', 'clients', 'employers'], index=['dateTime'])
minutes_left = int((date_time_end - date_time_i).seconds / 60)
pbar = tqdm(desc = 'Minutes', total=minutes_left)
while (date_time_i < date_time_end):
  if date_time_i.weekday() < 5:
    for office in df_offices['salePointName']:
      minutes = date_time_i.hour * 60 + date_time_i.minute
      if date_time_i.hour <= 11:
        koef = (df_offices[df_offices['salePointName'] == office]['daily_peak_k'] - \
                df_offices[df_offices['salePointName'] == office]['morning_k']) * \
                ((minutes - 480) / 180) + df_offices[df_offices['salePointName'] == office]['morning_k']
      elif date_time_i.hour <= 13:
        koef = (df_offices[df_offices['salePointName'] == office]['dinner_k'] - \
                df_offices[df_offices['salePointName'] == office]['daily_peak_k']) * \
                ((minutes - 660) / 120) + df_offices[df_offices['salePointName'] == office]['daily_peak_k']
      elif date_time_i.hour <= 15:
        koef = (df_offices[df_offices['salePointName'] == office]['evening_peak_k'] - \
                df_offices[df_offices['salePointName'] == office]['dinner_k']) * \
                ((minutes - 780) / 120) + df_offices[df_offices['salePointName'] == office]['dinner_k']
      elif date_time_i.hour <= 20:
        koef = (df_offices[df_offices['salePointName'] == office]['evining_k'] - \
                df_offices[df_offices['salePointName'] == office]['evening_peak_k']) * \
                ((minutes - 900) / 300) + df_offices[df_offices['salePointName'] == office]['evening_peak_k']
      else:
        koef = 0
      if date_time_i.hour <= 20:
        clients_now = int(df_offices[df_offices['salePointName'] == office]['employers']) * 1.3
        clients_now += (koef / 100) * clients_now
        clients_now = int(clients_now)
        employers_now = int(df_offices[df_offices['salePointName'] == office]['employers'])
        one_row(date_time_i, office, True, clients_now, employers_now)
      else:
        date_time_i += datetime.timedelta(hours=12)
        print('day', date_time_i.day, 'finished')
  else:
    date_time_i += datetime.timedelta(days=2)
  date_time_i += datetime.timedelta(minutes=10)
  pbar.update(10)
pbar.close()

Minutes:   0%|          | 0/960 [00:00<?, ?it/s]

day 4 finished
day 5 finished
day 6 finished
day 7 finished
day 8 finished
day 11 finished
day 12 finished
day 13 finished
day 14 finished
day 15 finished
day 18 finished


In [41]:
df_loading

,dateTime,salePointName,status,clients,employers
0,2022-01-18 09:00:00,ДО «Солнечногорский» Филиала № 7701 Банка ВТБ ...,True,11,9
1,2022-01-18 09:00:00,ДО «На Баранова» Филиала № 7701 Банка ВТБ (ПАО),True,23,18
2,2022-01-18 09:00:00,ДО «Лобня» Филиала № 7701 Банка ВТБ (ПАО),True,25,21
3,2022-01-18 09:00:00,ДО «Черноголовский» Филиала № 7701 Банка ВТБ (...,True,17,14
4,2022-01-18 09:00:00,ДО «Пушкинский» Филиала № 7701 Банка ВТБ (ПАО),True,16,13
...,...,...,...,...,...
220754,2022-01-03 08:00:00,ДО «Центральный» в г. Подольске Филиала № 7701...,True,15,13
220755,2022-01-03 08:00:00,ДО «Московский» в г. Московском Филиала № 7701...,True,22,20
220756,2022-01-03 08:00:00,ДО «Бронницкий» Филиала № 7701 Банка ВТБ (ПАО),True,23,20
220757,2022-01-03 08:00:00,ДО «ЗИО Подольск» Филиала № 7701 Банка ВТБ (ПАО),True,9,8


In [42]:
df_loading.to_csv('./drive/MyDrive/vtb24_hktn/loadings.csv')

In [ ]:
for office in df_offices['salePointName']:
  print(office)
  print(int(df_offices[df_offices['salePointName'] == office]['employers']))

ДО «ЗИО Подольск» Филиала № 7701 Банка ВТБ (ПАО)
5
ДО «Бронницкий» Филиала № 7701 Банка ВТБ (ПАО)
5
ДО «Московский» в г. Московском Филиала № 7701 Банка ВТБ (ПАО)
14
ДО «Центральный» в г. Подольске Филиала № 7701 Банка ВТБ (ПАО)
16
ДО «Подольский» Филиала № 7701 Банка ВТБ (ПАО)
14
ДО «На Каширском» Филиала № 7701 Банка ВТБ (ПАО)
6
ДО «Троицк» Филиала № 7701 Банка ВТБ (ПАО)
13
ДО «Видновский» Филиала № 7701 Банка ВТБ (ПАО)
17
ДО «На Венёвской» Филиала № 7701 Банка ВТБ (ПАО)
5
ДО «Бульвар Ушакова» Филиала № 7701 Банка ВТБ (ПАО)
15
ДО «Видное» Филиала № 7701 Банка ВТБ (ПАО)
11
ДО «Бутовский» Филиала № 7701 Банка ВТБ (ПАО)
21
ДО «Раменский» Филиала № 7701 Банка ВТБ (ПАО)
17
ДО «На Красноармейской» Филиала № 7701 Банка ВТБ (ПАО)
23
ДО «Жуковский Центральный» Филиала № 7701 Банка ВТБ (ПАО)
20
ДО «Россошанский» Филиала № 7701 Банка ВТБ ПАО)
24
ДО «На Янгеля» Филиала № 7701 Банка ВТБ (ПАО)
18
ДО «Жуковский» Филиала № 7701 Банка ВТБ (ПАО)
10
ДО «Ясенево» Филиала № 7701 Банка ВТБ (ПАО)
13
ДО «Аэ

TypeError: ignored

In [ ]:
offices_excel.drop(offices_excel.index[offices_excel['sale_point_name'] == 'Тест6'], inplace=True)

In [ ]:
df_offices[df_offices['salePointName'] == 'ДО «Люберецкий» Филиала № 7701 Банка ВТБ (ПАО)']['employers']

211    11
230    20
Name: employers, dtype: object